# Load Dataset

In [33]:
!pip install -q datasets
!pip install dahuffman
!pip install transformers
!pip install fpzip

In [34]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import ViTModel
import numpy as np
import torch
import cv2
import torch.nn as nn
from transformers import ViTModel, ViTConfig
from torchvision import transforms
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import numpy as np
from collections import Counter
import torch
from collections import Counter
from dahuffman import HuffmanCodec
import fpzip


In [3]:
dataset = load_dataset("Matthijs/snacks")
print(dataset)
print(dataset["train"].features['label'].names)

labels = dataset["train"].features["label"].names
num_labels = len(dataset["train"].features["label"].names)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
print(label2id)
print(id2label)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 4838
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 952
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 955
    })
})
['apple', 'banana', 'cake', 'candy', 'carrot', 'cookie', 'doughnut', 'grape', 'hot dog', 'ice cream', 'juice', 'muffin', 'orange', 'pineapple', 'popcorn', 'pretzel', 'salad', 'strawberry', 'waffle', 'watermelon']
{'apple': 0, 'banana': 1, 'cake': 2, 'candy': 3, 'carrot': 4, 'cookie': 5, 'doughnut': 6, 'grape': 7, 'hot dog': 8, 'ice cream': 9, 'juice': 10, 'muffin': 11, 'orange': 12, 'pineapple': 13, 'popcorn': 14, 'pretzel': 15, 'salad': 16, 'strawberry': 17, 'waffle': 18, 'watermelon': 19}
{0: 'apple', 1: 'banana', 2: 'cake', 3: 'candy', 4: 'carrot', 5: 'cookie', 6: 'doughnut', 7: 'grape', 8: 'hot dog', 9: 'ice cream', 10: 'juice', 11: 'muffin', 12: 'orange', 13: 'pineapple', 14: 'popcorn', 15: 'pretzel'

# Forward Pass ViT Model with a Toy Image

In [4]:
# Create toy image with dim (batch x channel x width x height)
toy_img = torch.rand(1, 3, 48, 48)

# Define conv layer parameters
num_channels = 3
hidden_size = 768
patch_size = 16

projection = nn.Conv2d(num_channels, hidden_size, kernel_size=patch_size,
             stride=patch_size)

out_projection = projection(toy_img)

print(f'Original image size: {toy_img.size()}')
print(f'Size after projection: {out_projection.size()}')


patch_embeddings = out_projection.flatten(2).transpose(1, 2)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define [CLS] token embedding with the same emb dimension as the patches
batch_size = 1
cls_token = nn.Parameter(torch.randn(1, 1, hidden_size))
cls_tokens = cls_token.expand(batch_size, -1, -1)

# Prepend [CLS] token in the beginning of patch embedding
patch_embeddings = torch.cat((cls_tokens, patch_embeddings), dim=1)
print(f'Patch embedding size: {patch_embeddings.size()}')

# Define position embedding with the same dimension as the patch embedding
position_embeddings = nn.Parameter(torch.randn(batch_size, 10, hidden_size))

# Add position embedding into patch embedding
input_embeddings = patch_embeddings + position_embeddings
print(f'Input embedding size: {input_embeddings.size()}')


# Define parameters for ViT-base (example)
num_heads = 12
num_layers = 12

# Define Transformer encoders' stack
transformer_encoder_layer = nn.TransformerEncoderLayer(
           d_model=hidden_size, nhead=num_heads,
           dim_feedforward=int(hidden_size * 4),
           dropout=0.1)
transformer_encoder = nn.TransformerEncoder(
           encoder_layer=transformer_encoder_layer,
           num_layers=num_layers)

# Forward pass
output_embeddings = transformer_encoder(input_embeddings)
print(f' Output embedding size: {output_embeddings.size()}')

# Load pretrained model
model_checkpoint = 'google/vit-base-patch16-224-in21k'
model = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)

# Example input image
input_img = torch.rand(batch_size, num_channels, 224, 224)

# Forward pass input image
output_embedding = model(input_img)
print(output_embedding)
print(f"Ouput embedding size: {output_embedding['last_hidden_state'].size()}")

num_labels = 20

# Define linear classifier layer
classifier = nn.Linear(hidden_size, num_labels)

# Forward pass on the output embedding of [CLS] token
output_classification = classifier(output_embedding['last_hidden_state'][:, 0, :])
print(f"Output embedding size: {output_classification.size()}")

Original image size: torch.Size([1, 3, 48, 48])
Size after projection: torch.Size([1, 768, 3, 3])
Patch embedding size: torch.Size([1, 9, 768])
Patch embedding size: torch.Size([1, 10, 768])
Input embedding size: torch.Size([1, 10, 768])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


 Output embedding size: torch.Size([1, 10, 768])


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.0233, -0.2161,  0.0567,  ...,  0.1944,  0.0654, -0.2822],
         [ 0.2157, -0.0004,  0.0636,  ..., -0.0062,  0.1340, -0.2078],
         [-0.1086, -0.3098,  0.0064,  ...,  0.1278,  0.3069, -0.3017],
         ...,
         [-0.1473, -0.2887, -0.0789,  ...,  0.0519,  0.1420, -0.1957],
         [ 0.0058, -0.1298, -0.0214,  ...,  0.0573,  0.0627, -0.2834],
         [ 0.0277, -0.1522, -0.0321,  ..., -0.0613,  0.0902, -0.0716]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=None, hidden_states=None, attentions=None)
Ouput embedding size: torch.Size([1, 197, 768])
Output embedding size: torch.Size([1, 20])


# Fine Tuning ViT Impelementation

In [5]:
#Pretrained model checkpoint
model_checkpoint = 'google/vit-base-patch16-224-in21k'

In [6]:
class ImageDataset(torch.utils.data.Dataset):

  def __init__(self, input_data):

      self.input_data = input_data
      self.transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
        ])

  def __len__(self):
      return len(self.input_data)

  def get_images(self, idx):
      return self.transform(self.input_data[idx]['image'])

  def get_labels(self, idx):
      return self.input_data[idx]['label']

  def __getitem__(self, idx):
      # Get input data in a batch
      train_images = self.get_images(idx)
      train_labels = self.get_labels(idx)

      return train_images, train_labels

In [55]:
def train_model(model, train_dataset, epochs, learning_rate, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    train_loader = DataLoader(ImageDataset(train_dataset), batch_size=batch_size, shuffle=True, num_workers=1)
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        total_loss, total_acc = 0, 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

        avg_loss = total_loss / len(train_loader.dataset)
        avg_acc = total_acc / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return model

def test_model(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    model = model.to(device)

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

In [8]:
import torch
import torch.nn as nn
from transformers import ViTModel, ViTConfig

class ViT(nn.Module):
    def __init__(self, config=ViTConfig(), num_labels=20,
                 model_checkpoint='google/vit-base-patch16-224-in21k',
                 encoded_weights=None, codec=None):
        super(ViT, self).__init__()
        self.vit = ViTModel.from_pretrained(model_checkpoint, add_pooling_layer=False)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.encoded_weights = encoded_weights
        self.codec = codec

    def forward(self, x):
        x = self.vit(x)['last_hidden_state']
        output = self.classifier(x[:, 0, :])
        return output


In [9]:
# Hyperparameters
EPOCHS = 3
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")



In [51]:
model = ViT().to(device)
trained_model = train_model(model, dataset['train'], EPOCHS, LEARNING_RATE, BATCH_SIZE)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 152/152 [02:33<00:00,  1.01s/it]


Epoch 1/3 - Loss: 0.0441, Accuracy: 0.8332


100%|██████████| 152/152 [02:32<00:00,  1.01s/it]


Epoch 2/3 - Loss: 0.0091, Accuracy: 0.9704


100%|██████████| 152/152 [02:33<00:00,  1.01s/it]

Epoch 3/3 - Loss: 0.0036, Accuracy: 0.9903


In [98]:
# load weights if you have it locally so don't have to fine tune again

# trained_model =  ViT()
#trained_model.load_state_dict(torch.load('original_model.pth'))

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
test_loss, test_accuracy = test_model(trained_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:10<00:00,  2.83it/s]

Test - Loss: 0.0151, Accuracy: 0.8792
0.015100919218937388 0.8792016806722689


In [57]:
ViT()

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

# Round model weights

In [13]:
import copy


trained_model.eval()  # Set the model to evaluation mode
model.to('cpu')  # Move the model to CPU

rounded_model = copy.deepcopy(trained_model)
rounded_model.to('cpu')
precision = 2

def round_with_decimal_precision(tensor, decimal_precision=2):
    # Convert the PyTorch tensor to a NumPy array
    numpy_array = tensor.numpy()

    # Apply rounding with decimal precision using NumPy
    rounded_numpy_array = np.round(numpy_array, decimals=decimal_precision)

    # Convert the rounded NumPy array back to a PyTorch tensor
    rounded_tensor = torch.tensor(rounded_numpy_array, dtype=tensor.dtype)

    return rounded_tensor


for name, param in rounded_model.named_parameters():
    param.data = round_with_decimal_precision(param.data, decimal_precision=precision).to(torch.float16)
    print(param)
    print(param.dtype)


Streaming output truncated to the last 5000 lines.
Parameter containing:
tensor([ 0.1300,  0.1801, -0.1000,  0.0300, -0.0000, -0.0200,  0.0200, -0.1000,
         0.0200, -0.1400, -0.0100,  0.0000, -0.0400,  0.0200, -0.0500, -0.1100,
         0.0800,  0.0300,  0.0900,  0.0700, -0.0400,  0.0800, -0.0000, -0.0900,
        -0.0500,  0.0400, -0.0300,  0.0100,  0.0300, -0.1000,  0.0700, -0.0100,
        -0.1100, -0.1100, -0.0700,  0.0800,  0.0900,  0.1000, -0.0400, -0.0500,
         0.0400,  0.0200,  0.0400,  0.1000,  0.0800,  0.0200, -0.1500, -0.1200,
         0.0200,  0.0300,  0.0500,  0.2500, -0.0600, -0.0100,  0.0300,  0.0100,
        -0.0700, -0.0100, -0.0500,  0.0400,  0.0600,  0.0500,  0.0200,  0.1700,
         0.0700, -0.0200, -0.0500, -0.0200,  0.0200,  0.1000, -0.0500,  0.0000,
        -0.0300,  0.0300,  0.0800, -0.0400,  0.0100, -0.1000, -0.0600, -0.0500,
        -0.0500,  0.0500,  0.0100,  0.0500,  0.0000,  0.1000, -0.2800,  0.0200,
        -0.0100, -0.0500,  0.0900, -0.0900, -0.

In [14]:
print("Original Weights:")
for param in trained_model.parameters():
    print(param.data)
    print(param.dtype)

Streaming output truncated to the last 5000 lines.
         4.9565e-02,  3.8935e-03,  3.4199e-02, -1.8823e-01, -3.8818e-02,
        -7.1279e-03, -4.6336e-02,  3.6418e-02,  3.6841e-02,  2.1763e-02,
        -4.1612e-03, -1.5075e-02, -2.9698e-02,  4.1186e-02, -9.1531e-02,
         9.4003e-02,  4.1221e-02,  2.3346e-02,  5.3583e-02,  1.3995e-02,
        -5.2272e-02, -3.4372e-02, -1.3051e-02, -4.5918e-02,  1.9390e-02,
         1.7093e-02, -3.4043e-02,  2.7600e-02, -2.0013e-02, -9.8622e-02,
         1.3843e-01,  6.6632e-03, -5.3812e-02,  1.8786e-02, -5.1360e-02,
        -9.2917e-02,  5.4448e-02,  2.6478e-02, -3.8288e-02,  1.4716e-02,
         5.3663e-02,  2.5523e-02, -2.3143e-02,  3.0305e-02,  7.7394e-03,
        -3.8964e-02, -2.4574e-02,  2.1202e-02,  3.2504e-02, -5.7624e-02,
         9.6581e-03,  4.2131e-02,  5.4521e-02,  1.8373e-01, -1.7239e-02,
         5.5846e-02, -2.4548e-02,  1.3799e-02,  1.1451e-02, -1.1025e-01,
         1.2107e-02, -5.8300e-02, -6.1996e-02,  2.3969e-02, -4.2913e-02,


In [15]:
def test_model_rounded(model, test_dataset, batch_size):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model.eval()
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc

# Test the model
test_loss, test_accuracy = test_model_rounded(rounded_model, dataset['test'], BATCH_SIZE)
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:04<00:00,  6.69it/s]

Test - Loss: 0.0109, Accuracy: 0.9149
0.010887081883534664 0.9149159663865546


In [16]:
import torch
import os
model.to("cpu")
# Assuming 'model' is your original ViT model and 'quantized_model' is the quantized version

# Save the original model
torch.save(trained_model.state_dict(), 'original_model.pth')

# Save the quantized model
torch.save(rounded_model.state_dict(), 'rounded_model.pth')

# Function to get file size in MB
def file_size_in_mb(file_path):
    size_bytes = os.path.getsize(file_path)
    return size_bytes / (1024 * 1024)  # Convert bytes to megabytes

# Print the file sizes
original_size = file_size_in_mb('original_model.pth')
rounded_model_size = file_size_in_mb('rounded_model.pth')
print(f"Original Model Size: {original_size:.2f} MB")
print(f"Rounded Model Size: {rounded_model_size:.2f} MB")

Original Model Size: 327.43 MB
Rounded Model Size: 163.76 MB


# Define LZW encoding

In [32]:
import torch
import struct
import torch
import struct
import io



# The original float value is 123.456 -> its 4-byte representation is a byte string b'y\xe9\xf6B'.
def serialize_weights_pytorch(model):
    serialized_weights = []
    for param_tensor in model.state_dict():
        tensor = model.state_dict()[param_tensor].cpu().numpy()
        flattened_weights = tensor.flatten()
        for weight in flattened_weights:
            # Convert each weight to a 4-byte representation
            serialized_weights.append(struct.pack('f', weight))
    return b''.join(serialized_weights)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.1 MB/s eta 0:00:00


In [23]:
# Updated LZW compression function to output a byte stream
# def lzw_compress(uncompressed):
#     dict_size = 256
#     dictionary = {bytes([i]): i for i in range(dict_size)}

#     w = b""
#     result = io.BytesIO()
#     for c in uncompressed:
#         wc = w + bytes([c])
#         if wc in dictionary:
#             w = wc
#         else:
#             result.write(dictionary[w].to_bytes(2, byteorder='little'))
#             dictionary[wc] = dict_size
#             dict_size += 1
#             w = bytes([c])

#     if w:
#         result.write(dictionary[w].to_bytes(2, byteorder='little'))
#     return result.getvalue()



def lzw_compress(uncompressed):
    """Compress a string to a list of output symbols."""

    # Build the dictionary.
    dict_size = 256
    dictionary = dict((chr(i), i) for i in range(dict_size))
    # in Python 3: dictionary = {chr(i): i for i in range(dict_size)}

    w = ""
    result = []
    for c in uncompressed:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            # Add wc to the dictionary.
            dictionary[wc] = dict_size
            dict_size += 1
            w = c

    # Output the code for w.
    if w:
        result.append(dictionary[w])
    return result


def lzw_decompress(compressed):
    """Decompress a list of output ks to a string."""
    from io import StringIO

    # Build the dictionary.
    dict_size = 256
    dictionary = dict((i, chr(i)) for i in range(dict_size))
    # in Python 3: dictionary = {i: chr(i) for i in range(dict_size)}

    # use StringIO, otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = StringIO()
    w = chr(compressed.pop(0))
    result.write(w)
    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        result.write(entry)

        # Add w+entry[0] to the dictionary.
        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry
    return result.getvalue()

In [104]:
def compress_params(model):
    compressed_state_dict = {}
    for param_name in model.state_dict():
        result_string = ""
        print(param_name)
        # serialized_weights=[]
        tensor = model.state_dict()[param_name].cpu().numpy()
        flattened_weights = tensor.flatten()
        result_string = ''.join(map(str, flattened_weights))
        # print(result_string)
        # for weight in flattened_weights:
        # #     print(struct.pack('f', weight))
        # #     # Convert each weight to a 4-byte representation
        #     serialized_weights.append(struct.pack('f', weight))

        compressed_weights = lzw_compress(result_string)
        # print(lzw_decompress(compressed_weights))
        compressed_state_dict[param_name] = compressed_weights

    return compressed_state_dict

# def compress_params(model):
#     compressed_state_dict = {}
#     for param_name in model.state_dict():
#         result_string = ""
#         print(param_name)
#         # serialized_weights=[]
#         tensor = model.state_dict()[param_name].cpu().numpy()
#         flattened_weights = tensor.flatten()
#         # compressed_weights = fpzip.compress(flattened_weights, precision=0, order='F')
#         # print(lzw_decompress(compressed_weights))
#         compressed_state_dict[param_name] = compressed_weights

#     return compressed_state_dict

copy=rounded_model


In [105]:
compressed_state_dict = compress_params(copy)


vit.embeddings.cls_token
<class 'numpy.ndarray'>
vit.embeddings.position_embeddings
<class 'numpy.ndarray'>
vit.embeddings.patch_embeddings.projection.weight
<class 'numpy.ndarray'>
vit.embeddings.patch_embeddings.projection.bias
<class 'numpy.ndarray'>
vit.encoder.layer.0.attention.attention.query.weight
<class 'numpy.ndarray'>
vit.encoder.layer.0.attention.attention.query.bias
<class 'numpy.ndarray'>
vit.encoder.layer.0.attention.attention.key.weight
<class 'numpy.ndarray'>
vit.encoder.layer.0.attention.attention.key.bias
<class 'numpy.ndarray'>
vit.encoder.layer.0.attention.attention.value.weight
<class 'numpy.ndarray'>
vit.encoder.layer.0.attention.attention.value.bias
<class 'numpy.ndarray'>
vit.encoder.layer.0.attention.output.dense.weight
<class 'numpy.ndarray'>
vit.encoder.layer.0.attention.output.dense.bias
<class 'numpy.ndarray'>
vit.encoder.layer.0.intermediate.dense.weight
<class 'numpy.ndarray'>
vit.encoder.layer.0.intermediate.dense.bias
<class 'numpy.ndarray'>
vit.encode

In [106]:
torch.save(compressed_state_dict, 'fpzip_compressed_model.pth')

In [107]:
lzw_compressed_size = file_size_in_mb('fpzip_compressed_model.pth')
print(f"FPZIP Model Size: {lzw_compressed_size:.2f} MB")

FPZIP Model Size: 15.53 MB


# Huffman encoding

In [ ]:
# Collect encoding statistics


device = torch.device("cpu")
rounded_model.to(device)


all_weights_flat = []


for name, param in rounded_model.named_parameters():
    layer_weights = param.data.cpu().numpy().flatten().tolist()
    all_weights_flat.extend(layer_weights)


frequencies = Counter(all_weights_flat)
codec = HuffmanCodec.from_frequencies(frequencies)

print(frequencies)

In [ ]:
huffman_model = copy.deepcopy(rounded_model)
encoded_weights_dict = {}

for name, param in huffman_model.named_parameters():
    layer_weights = param.data.flatten().tolist()
    encoded_weights = codec.encode(layer_weights)
    encoded_weights_dict[name] = encoded_weights
    decoded_weights = codec.decode(encoded_weights)
    assert layer_weights == decoded_weights, "Mismatch in encoding/decoding!"

torch.save(encoded_weights_dict, 'encoded_model_weights.pth')

In [ ]:

huffman_size = file_size_in_mb('encoded_model_weights.pth')
print(f"Huffman Model Size: {huffman_size:.2f} MB")

In [ ]:
for k in iter(encoded_weights_dict):
  print(k)

# Test model

In [108]:
def test_model_lzw(model, test_dataset, batch_size, encoded_weights_file):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    test_loader = DataLoader(ImageDataset(test_dataset), batch_size=batch_size, shuffle=False, num_workers=1)

    model = model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    total_loss, total_acc = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total_acc += (outputs.argmax(dim=1) == labels).sum().item()

    avg_loss = total_loss / len(test_loader.dataset)
    avg_acc = total_acc / len(test_loader.dataset)
    print(f'Test - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')

    return avg_loss, avg_acc, model.state_dict()

## Test entire decoded LZW model

In [111]:
encoded_weights_dict = torch.load('lzw_compressed_model.pth')
test_model = ViT()
test_model.to(device)

for name, param in model.named_parameters():
    print(name)
    if name in encoded_weights_dict:
      # decoded_weights = fpzip.decompress(encoded_weights_dict[name], order='C')
        decoded_weights = lzw_decompress(encoded_weights_dict[name])
        string_list_of_values = decoded_weights.split(',')
        float_list_of_values = []
        for value in string_list_of_values:
          # print(value)
          float_list_of_values.append(np.float16(value))
        decoded_weights_tensor = torch.tensor(float_list_of_values, device=device, dtype=torch.float16).view_as(param.data)
        param.data.copy_(decoded_weights_tensor)
    else:
        print(f"Weight for layer {name} not found in encoded weights.")

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vit.embeddings.cls_token
vit.embeddings.position_embeddings
vit.embeddings.patch_embeddings.projection.weight
vit.embeddings.patch_embeddings.projection.bias
vit.encoder.layer.0.attention.attention.query.weight
vit.encoder.layer.0.attention.attention.query.bias
vit.encoder.layer.0.attention.attention.key.weight
vit.encoder.layer.0.attention.attention.key.bias
vit.encoder.layer.0.attention.attention.value.weight
vit.encoder.layer.0.attention.attention.value.bias
vit.encoder.layer.0.attention.output.dense.weight
vit.encoder.layer.0.attention.output.dense.bias
vit.encoder.layer.0.intermediate.dense.weight
vit.encoder.layer.0.intermediate.dense.bias
vit.encoder.layer.0.output.dense.weight
vit.encoder.layer.0.output.dense.bias
vit.encoder.layer.0.layernorm_before.weight
vit.encoder.layer.0.layernorm_before.bias
vit.encoder.layer.0.layernorm_after.weight
vit.encoder.layer.0.layernorm_after.bias
vit.encoder.layer.1.attention.attention.query.weight
vit.encoder.layer.1.attention.attention.query

In [112]:
test_model.eval()
test_loss, test_accuracy, state_dic = test_model_lzw(test_model, dataset['test'], BATCH_SIZE, 'encoded_model_weights.pth')
print(test_loss, test_accuracy)

100%|██████████| 30/30 [00:11<00:00,  2.68it/s]

Test - Loss: 0.0942, Accuracy: 0.0609
0.09418207181601965 0.06092436974789916


## Test dynamically decoded lzw model